In [5]:
import tensorflow as tf
config=tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.InteractiveSession(config=config)

# Static and Dynamic Shape
Create a placeholder with static shape (None, 10). We can get that shape using the `shape` method. The dynamic shape is available by `tf.shape`. The static shape is a python value that may contain undefined values whereas the dynamic shape is a Tensor (so each value has to be specified) that only gets an actual value during computations. 

In [8]:
a = tf.placeholder(dtype=tf.float32, shape=(None, 1))
print(a.shape)
dynamic_shape = tf.shape(a)
print(dynamic_shape)

(?, 1)
Tensor("Shape_1:0", shape=(2,), dtype=int32)


We can get the shape tensor as we do any other value from the graph. It depends on the actual shapes that we put into the computation.

In [14]:
print(dynamic_shape.eval(feed_dict={a: [[1], [2], [3]]}))
print(dynamic_shape.eval(feed_dict={a: [[1], [2], [3], [4]]}))

[3 1]
[4 1]


The static shape need not be a python literal. It is static in a single computation graph, not necessarily a constant in the python script.

In [30]:
BATCH_SIZE = 32  # normally, get this from the command line
b = tf.placeholder(dtype=tf.float32, shape=(BATCH_SIZE, 1))
print(b)

Tensor("Placeholder_9:0", shape=(32, 1), dtype=float32)


A shape can also be completely unspecified or only be defined up to its rank. This happens for example in operations that read images from files. In this case we can manually refine the shape. This is useful because it improves consistency checking of the computation graph, and somtimes necessary as certain operations need to know the (partial) shape of the data they operate on. In that case we can use `set_shape`  to add an additional constraint to the shape. This means that `set_shape` can only make the shape more specific, and has no influence on the dynamic shape.

In [42]:
image = tf.image.decode_png(tf.placeholder(dtype=tf.string))
print(image.shape)
image.set_shape((None, None, 3))
print(image.shape)
image.set_shape((64, 64, None))
print(image.shape)

(?, ?, ?)
(?, ?, 3)
(64, 64, 3)


# Violating Shape Constraints

But any dynamic value we want to put in has to be compatible with the static shape. Compatible means that the need the same rank (if specified) and same size in each dimension where the static shape is specified. The following will fail because the value we want to feed is not compatible with the placeholder's shape.

In [ ]:
print(dynamic_shape.eval(feed_dict={a: [1, 2, 3]}))

Sometimes, the incompatibility only appears later in the calculations.

In [26]:
b = tf.zeros(dtype=tf.float32, shape=(1, 5))
c = tf.add(a, b)
print(c.shape)

(2, 5)


In [ ]:
print(c.eval(feed_dict={a: [[1], [2], [3]]}))

It is not possible to change the shape specification to something that is incompatible with the current specification.

In [43]:
c.set_shape((50, 50))

ValueError: Shapes (2, 5) and (50, 50) are not compatible

# Broadcasting
Tensors of different shapes can be added (multiplied, ...) together if they can be broadcast to a common shape. The rules are the same as in numpy (at least I think so). Dimensions of size one can be broadcasted to the corresponding dimension of the other tensor. Missing dimensions are all considered to be of size one.

In [44]:
tf.zeros((10, 1)) + tf.zeros((1, 10))

<tf.Tensor 'add_2:0' shape=(10, 10) dtype=float32>

In [46]:
tf.zeros((10)) + tf.zeros((1, 10))

<tf.Tensor 'add_4:0' shape=(1, 10) dtype=float32>

In [47]:
tf.zeros((10, 1)) + tf.zeros((10))

<tf.Tensor 'add_5:0' shape=(10, 10) dtype=float32>

# Reshaping
The dynamic tensor shape can be changed using `tf.reshape`. Even though the dynamic shapes have to be fully specified, you can leave at most one dimension unspecified, which will be calculated based one the total size. 

_ Since `tf.reshape` operates on the dynamic shape, the target shape has to be a Tensor, so the unspecified dimension has to be marked with -1 instead of None. _

In [54]:
a = tf.placeholder(tf.float32, (None, 10, 10))
b = tf.reshape(a, (-1, 100))
print(b)

Tensor("Reshape_1:0", shape=(?, 100), dtype=float32)


We can use a completely dynamical value for the new shape. In this case, `b` won't have any shape information left.

In [57]:
shape = tf.placeholder(tf.int32)
b = tf.reshape(a, shape)
print(b)

Tensor("Reshape_3:0", dtype=float32)


If the dynamical value for the shape itself has a specified length, we can at least infer the rank of the new shape. If the new shape tensor is not compatible to rank one, reshape won't accept it.

In [60]:
shape = tf.placeholder(tf.int32, shape=(2))
b = tf.reshape(a, shape)
print(b)

Tensor("Reshape_6:0", shape=(?, ?), dtype=float32)
